# Hourly Traffic Analytics for 6 Camps

### TODOs
- convert Active Time to hours
- create visualization with 2 y-axes including: 1.# of devices and 2.Active Time in hours/minutes, x-axis is 2hr time chunks

In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import requests

#--For importing Files--
import glob
import os

#--For Meraki--
from meraki import meraki
import json
from ipaddress import ip_address
import re
import warnings

In [54]:
file = 'Data/even_hourly_traffic_analytics_Apr_four_thru_five/*.csv'

In [3]:
camp_lst = ['Moria', 'Nea', 'Vathi', 'Skaram', 'Kos', 'Veria']
#     date_lst = ['040519', '040619', '040719', '040819', '040919', '041019', '041119']

globbed_files = glob.glob(file) #create a list of all csv files
data = [] 
for file in globbed_files:

    frame = pd.read_csv(file)
    frame['filename'] = os.path.basename(file)
#         frame['date'] = date
#         frame['Time'] = time    
    data.append(frame)

app_usage = pd.concat(data, ignore_index=True)

    
#--Add camp name-
camp_list = list(app_usage['filename'].unique())
s = '(' + '|'.join(camp_lst) + ')'
app_usage['camp_name'] = app_usage['filename'].str.extract(s, expand=False)

app_usage['hour'] = 0

app_usage['hour'] = app_usage['filename'].str.extract("(\d+)", expand=True)



# df["B"] = df["A"].str.extract("(\d*\.?\d+)", expand=True)

# m = app_usage['filename'].extract('(\d+)')
# app_usage['hour'] = app_usage['filename'].str.extract(r'(\d+)')


# app_usage['hour'] = app_usage['filename'].str.extract('(\d+)', expand=False)
# data['active_time_num'] = data['Active time'].str.extract('(\d+)')

# #--Add time--
# date_list = list(app_usage['filename'].unique())
# s = '(' + '|'.join(date_lst) + ')'
# app_usage['date'] = app_usage['filename'].str.extract(s, expand=False)
# app_usage['date'] = app_usage['date'].astype(str).astype(int)

# globbed_files

In [56]:
len(globbed_files)

72

In [2]:
data = app_usage.copy()
data

In [ ]:
def make_df(files):
    camp_lst = ['Moria', 'Nea', 'Vathi', 'Skaram', 'Kos', 'Veria']
#     date_lst = ['040519', '040619', '040719', '040819', '040919', '041019', '041119']
    
    globbed_files = glob.glob(files) #create a list of all csv files
    data = [] 
    for file in globbed_files:

        frame = pd.read_csv(file)
        frame['filename'] = os.path.basename(file)
#         frame['date'] = date
#         frame['Time'] = time    
        data.append(frame)
    
    app_usage = pd.concat(data, ignore_index=True)
    
    #--Add camp name-
    camp_list = list(app_usage['filename'].unique())
    s = '(' + '|'.join(camp_lst) + ')'
    app_usage['camp_name'] = app_usage['filename'].str.extract(s, expand=False)
    
    #--Add date--
    date_list = list(app_usage['filename'].unique())
    s = '(' + '|'.join(date_lst) + ')'
    app_usage['date'] = app_usage['filename'].str.extract(s, expand=False)
    app_usage['date'] = app_usage['date'].astype(str).astype(int)
    
    return app_usage

In [45]:
#--Clean Active time, give consistent values in minutes--
data['ActiveTime_Minutes'] = 0

#--get increment
data['active_time_increment'] = data['Active time'].str.replace('\d+', '') 
data['active_time_increment'] = data['active_time_increment'].str.replace('.', '')

#--get amount--
# try:
#     data['active_time_num'] = data['Active time'].str.extract('(\d+\.\d+)')
# except:


### -- TODO: Fix line below, it only gets int, need to get both int and float numbers--
data['active_time_num'] = data['Active time'].str.extract('(\d+)')


data['active_time_num'] = data['active_time_num'].astype(str).astype(int)
# data

#--Loop through this list--
unique_vals = list(data['active_time_increment'].unique())

#--to get to minutes--
hour = 60
day = 24*60
week = 7*day
month = 30.3*day
year = 365*day

#Source: https://stackoverflow.com/questions/33769860/pandas-apply-but-only-for-rows-where-a-condition-is-met
#Source: https://stackoverflow.com/questions/10715519/conditionally-fill-column-values-based-on-another-columns-value-in-pandas
hr = data[data['active_time_increment'].str.contains('hour')] 
dy = data[data['active_time_increment'].str.contains('day')]
wk = data[data['active_time_increment'].str.contains('week')]
mon = data[data['active_time_increment'].str.contains('month')] 

#--Add minutes for each row--
hr['ActiveTime_Minutes'] = hour*hr['active_time_num']
dy['ActiveTime_Minutes'] = day*dy['active_time_num']
wk['ActiveTime_Minutes'] = week*wk['active_time_num']
mon['ActiveTime_Minutes'] = month*mon['active_time_num']

traffic = pd.concat([hr,dy,wk,mon])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [1]:
# traffic